# Camera Calibration

In this notebook, I aim to get familiar with camera Calibration and how to use it to undistort images.

In [12]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# (points_per_row,points_per_colum)
points_per_row = 6
points_per_colum = 9
pattern_size = (points_per_row, points_per_colum)

In [19]:
# prepare object points
objp = np.zeros((points_per_row * points_per_colum, 3), np.float32)
objp[:, :2] = np.mgrid[0:points_per_row, 0:points_per_colum].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
objpoints = []  # 3D points in real-world space
imgpoints = []  # 2D points in image plane

# Get all JPEG files in the current directory
image_files = glob.glob('*.jpeg')

for image_file in image_files:
    # Read the image
    img = cv.imread(image_file)

    # Convert img to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv.findChessboardCorners(gray, pattern_size, None)

    if ret:
        # If corners are found, add object points and image points
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, pattern_size, corners2, ret)

        # wait a bit
        cv.waitKey(1000)

    # Display the img
    cv.imshow('Image', img)

    # Wait for key press
    if cv.waitKey(5) == ord('q'):
        break

# Release the capture and destroy windows
cv.destroyAllWindows()

In [33]:
# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# undistort image
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
cv.imwrite('calibresult.png', dst)

True

In [24]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.02130933251164214
